In [ ]:
"""
TITLE: C964 Computer Science CapStone
AUTHOR: James Mills
STUDENT ID: 000955354
DATE: January 2022
"""
import copy, itertools, random, csv, re, importlib, requests
from PIL import Image
from io import BytesIO
from IPython.display import display, HTML,clear_output
import ipywidgets as widgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mlxtend.preprocessing import TransactionEncoder
from PyScripts.HashTable import HashTable
from functools import reduce

display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
pd.set_option('mode.chained_assignment', None)  # turns off warning about slicing databases.
product_rules = pd.read_csv('Data/master_rules.csv')

In [ ]:
# %matplotlib widget
with open('Data/names.csv') as f:
    read = csv.reader(f)
    names = list(read)
with open('Data/transaction_data.csv') as g:
    reader = csv.reader(g)
    items = list(reader)

In [ ]:
te = TransactionEncoder()

In [ ]:
te_ary = te.fit(items).transform(items)

In [ ]:
df = pd.DataFrame(te_ary, columns=te.columns_)

In [ ]:
numbers = []
master_recommend_list = []  # holds a list of DataFrames
# plot_out = widgets.Output()
plot_out_1 = widgets.Image(value=bytes("HELP",'utf-8'),format='png',width=400,height=400,layout=widgets.Layout(align='center',justify_items='center'))
for label, content in df.items():
    numbers.append(df[label].sum())
column_names = list(df.columns)
item_frequency = {column_names[i] : numbers[i] for i in range(len(column_names))}

In [ ]:
'''
Required Functions for program below.
'''


def make_plot():
    # clear_output()
    # clear()
    # plot_out.clear_output()
    global master_recommend_list
    plot_list = master_recommend_list.copy(deep=True)
    ax = plot_list.plot(kind='bar',figsize=(8,8))
    for container in ax.containers:
        ax.bar_label(container)
    fig_pd = ax.get_figure()
    plt.savefig('Images/output.png')
    plt.close(fig_pd)
    image_1_1 = open('Images/output.png','rb')
    image_1_1_1 = image_1_1.read()
    plot_out_1.value=image_1_1_1
    image_1_1.close()
    # plot_out_1.value=""
    # plot_out_1.value="<img src='Images/output.png' alt='Plot'>"
    # plot_out.clear_output()
    # with plot_out:
    #     display(ax)
def stringify_frozen_set(x):
    """
    :param x: Takes a entry from a Pandas Dataframe.
    :return: Reformat antecedents and consequents columns to Python Strings for correct display
    and returns the updated String.
    """
    beginning = x.find("'") + 1
    end = x.rfind("'")
    data = x[beginning: end]
    data_remove_apostrophes = data.replace("'", '')
    new_data = data_remove_apostrophes.split(',')
    new_data_stripped = []
    for i in range(len(new_data)):
        new_data_stripped.append(copy.copy(new_data[i].strip()))
    if len(new_data_stripped) == 1:
        return str(new_data_stripped)
    else:
        return str([x for x in new_data_stripped])

def fix_frozen_set(x):
    """
    :param x: Takes a entry from a Pandas Dataframe.
    :return: Reformat antecedents and consequents columns as Frozen-sets for correct querying
    and returns the updated Dataframe.
    """
    beginning = x.find("'") + 1
    end = x.rfind("'")
    data = x[beginning: end]
    data_remove_apostrophes = data.replace("'", '')
    new_data = data_remove_apostrophes.split(',')
    new_data_stripped = []
    for i in range(len(new_data)):
        new_data_stripped.append(copy.copy(new_data[i].strip()))
    if len(new_data_stripped) == 1:
        return frozenset(new_data_stripped)
    else:
        return frozenset([x for x in new_data_stripped])

def recommend(z):
    """

    :param z: Takes a list of items.
    :return: Set of recommended items so duplicated are removed.
    """
    global product_rules, master_recommend_list
    master_query = None
    while True:
        try:
            master_query = product_rules[product_rules['antecedents'] == {z[0], z[1], z[2], z[3], z[4]}]
        except IndexError:
            pass
        if master_query is not None:
            break
        try:
            master_query = product_rules[product_rules['antecedents'] == {z[0], z[1], z[2], z[3]}]
        except IndexError:
            pass
        if master_query is not None:
            break
        try:
            master_query = product_rules[product_rules['antecedents'] == {z[0], z[1], z[2]}]
        except IndexError:
            pass
        if master_query is not None:
            break
        try:
            master_query = product_rules[product_rules['antecedents'] == {z[0], z[1]}]
        except IndexError:
            pass
        if master_query is not None:
            break
        try:
            master_query = product_rules[product_rules['antecedents'] == {z[0]}]
        except IndexError:
            return "OOPS!"
        if master_query is None:
            return None
    master_query.sort_values(by=['support', 'confidence', 'lift'], inplace=True, ascending=(False, False, False),
                             ignore_index=True)
    master_recommend_list = master_query.copy(deep=True)
    if not master_recommend_list.empty:
        make_plot()
    else:
        plot_out_1.value = bytes("HELP",'utf-8')
    formatted_master_query = master_query['consequents'].apply(lambda x: ', '.join(list(x))).astype("unicode")
    results = list(formatted_master_query)
    if len(results) == 0:
        return None
    else:
        return set(results)

def start_here(x):
    """

    :param x: List of items currently in Inventory
    to search for recommendations on.
    :return: A Python Set of recommendations based on the user's inventory.
    """
    master_search = []
    for j in range(1, 6):
        for element in itertools.combinations(x, j):
            master_search.append(list(element))
    while True:
        try:
            response = recommend(master_search[-1])
        except IndexError:
            return 'SOMETHING WENT WRONG!!!'
        if response is None or response == "OOPS!":
            master_search.pop()
            continue
        else:
            return response

In [ ]:
"""
Try block to ensure formatting of Dataframe is only completed once.
"""
try:
    # noinspection PyUnresolvedReferences
    if DATA_IS_LOADED:
        pass
except NameError:
    DATA_IS_LOADED = 1
    # Format panda dataframe for optimized queries
    product_rules['antecedents'] = product_rules['antecedents'].apply(fix_frozen_set)
    product_rules['consequents'] = product_rules['consequents'].apply(fix_frozen_set)


In [ ]:
dict_from_items = dict.fromkeys(item_frequency.copy().keys(), 0.0)
for i in range(len(dict_from_items)):
    dict_from_items.update({column_names[i] : ((item_frequency.get(column_names[i])) / 34766.0)})
new_list_1 = sorted(dict_from_items.items(), key=lambda kv:(kv[1], kv[0]))
top_ten = copy.copy(new_list_1[-1:-11:-1])
items_to_remove = [x[0] for x in copy.copy(top_ten)]
for i in range(len(items_to_remove)):
    dict_from_items.pop(items_to_remove[i])
other_item_total = list(dict_from_items.copy().values())
answer = sum(other_item_total)
others = ('All other items', answer)
top_ten.append(others)

In [ ]:
%%html
<h1 style="border: 5px dotted red;padding: 10px;font-size: 75px;text-align:center;">
Welcome to the presentation of the next generation of ShopperzЯUs shopping cart recommendation program!!!
</h1>

In [ ]:
%%html
<h2 style="text-align:center">Below you can see a few graphs demonstrating the percentages of customers who purchased a particular item. </h2>

In [ ]:
%%html
<h3 style="color:orange;font-size:150px;text-align:center;border:5px solid yellow"> &darr; &darr; &darr; &darr; &darr; &darr; &darr; &darr; &darr; &darr; &darr; &darr; &darr; &darr; &darr; </h3>

In [ ]:
color = plt.cm.rainbow(np.linspace(0, 1, 40))
x = item_frequency.keys()
y = item_frequency.values()
fig = plt.figure(figsize=(50,10))
plt.ylabel("Number of items purchased", fontsize = 24)
plt.title("Frequency of most purchased items", fontsize= 48)
plt.bar(x,y, color=color)
plt.xticks(rotation=90)
plt.show()
# fig.canvas.layout.min_width = '400px'
# fig.canvas.layout.flex = '1 1 auto'
# fig.canvas.layout.width = 'auto'
# fig.canvas.layout.object_fit = 'contain'

In [ ]:
colors = plt.get_cmap('Reds')(np.linspace(0.2, 0.5, len([x[0] for x in top_ten])))
fig1 = plt.figure(figsize=(25,25))
explode = (0,0,0,0,0,0,0,0,0,0,0.05)
patches, texts, autotexts = plt.pie([x[1] for x in top_ten], labels=[x[0] for x in top_ten], explode=explode, shadow=True, autopct='%1.2f%%')
plt.setp(autotexts, size='large')
plt.setp(texts, size='large', rotation=45)
plt.title("Top ten most purchased items", fontsize=24)
plt.show()
# fig1.canvas.layout.width = 'auto'

In [ ]:
%%html
<h3 style="text-align:center;color:blue;font-size: 50px;border: 5px solid gray; padding: 10px"> Here is a wordcloud format of the item-set data used. The larger the word, the more often it occurs in the transaction list. </h3>

In [ ]:
cloud = open('Images/wordcloud.png','rb')
image_1 = cloud.read()
cloud_show = widgets.Image(value=image_1,format='png',width=1600,height=800,layout=widgets.Layout(align='center',justify_items='center'))
display(cloud_show)

In [ ]:
%%html
<p>By utilizing the previous shoppers habits as training data, our new ShoppingKart app will display <b>real-time</b> recommendations for additional items to add to the user's inventory.</p>

In [ ]:
%%html
<p> This demonstration will show the recommendation portion of the app off. The development team at ShopperzЯUs are finishing up the Android based app for our customers to utilize daily for their grocery inventory, and shopping needs!! </p>

In [ ]:
%%html
<p> The item database is <strong>167</strong> unique items that can be added. The android app is going to process the input from the customer when creating their inventory to match up to the best item we have availble in order to make re-ordering and also upselling a breeze for the customer and profits for the company!! </p>

In [ ]:
%%html
<p> Below is a mock inventory builder. You can either select an item from the list, or add a random item from our inventory base to the inventory list along with the amount currently on hand. This system will dynamically generate both, the top recommended item(s) based on what has been entered, and will also generate the data associated behind the machine-learning based process of assosication that creates the recommondations via graphs. It also displays the confidence level for the recommendation that was made.</p>

In [ ]:
master_inventory = HashTable()
choices = widgets.Dropdown(options=names[0], value=names[0][0], description='Inventory Choices: ', disabled=False,style= {'description_width':'initial'})
random_choice = widgets.Button(description='Select Random Item', disabled=False, button_style='success',tooltip='CLICK HERE',icon='dice')
amount_choice = widgets.IntSlider(value=1,min=1,max=10,step=1,description='Inventory amount: ',disabled=False,orientation='horizontal'
                                  ,readout=True,readout_format='d',style= {'description_width':'initial'})
add_choice = widgets.Button(description='Add Selected Item', disabled=False, button_style='info', tooltip='ADD TO INV', icon='plus-square')
reset_inventory = widgets.Button(description='Reset Inventory to Zero', disabled=False, button_style='warning', tooltip='DESTROY INV', icon='bomb')
inventory_output = widgets.Output(layout={'border':'2px solid red'},disabled=False)
recommendation = widgets.HTML(value="NOTHING",description='Recommendations Show Here',style= {'description_width':'initial'})
output_label = widgets.Label(value='INVENTORY CONTENTS BELOW:')

In [ ]:
# def clear():
    # plot_out.clear_output()

def look_recommend(chh):
    random_choice.disabled = True
    reset_inventory.disabled = True
    add_choice.disabled = True
    orig_copy_received = set(copy.deepcopy(chh))
    value = start_here(chh)
    if value == 'SOMETHING WENT WRONG!!!':
        recommendation.value = "NOTHING"
        random_choice.disabled = False
        reset_inventory.disabled = False
        add_choice.disabled = False
    else:
        initial_recommend = set(value)
        initial_recommend.difference_update(orig_copy_received)
        if len(initial_recommend) == 0:
            recommendation.value = "NOTHING"
            random_choice.disabled = False
            reset_inventory.disabled = False
            add_choice.disabled = False
        else:
            recommendation.value = ""
            for i in range(len(initial_recommend)):
                vals = initial_recommend.pop()
                recommendation.value += "<strong>"+str(i+1)+"</strong>: " + vals + '<br>'
            random_choice.disabled = False
            reset_inventory.disabled = False
            add_choice.disabled = False

def rand(ch):
    index = random.randrange(len(names[0]))
    choices.value = str(names[0][index])
def destroy(ch1):
    inventory_output.clear_output()
    recommendation.value = 'NOTHING'
    global master_inventory
    master_inventory = None
    master_inventory = HashTable()
random_choice.on_click(rand)
reset_inventory.on_click(destroy)
def add_choice_to_inv(ch2):
    master_inventory.insert(str(choices.value), [str(choices.value), amount_choice.value])
    with inventory_output:
        inventory_output.clear_output()
        print(master_inventory)
    var = []
    for i in range(len(master_inventory.table)):
        if len(master_inventory.table[i]) > 0:
            var.append(copy.copy(master_inventory.table[i][0][0]))
    look_recommend(var)
add_choice.on_click(add_choice_to_inv)
# plot_out.on_msg(clear)
# plot_out.on_displayed(clear)
# plot_out.on_trait_change(clear)
# plot_out.observe(clear)

In [ ]:
grid = widgets.GridspecLayout(2,3,layout=widgets.Layout(width='auto', border='2px dotted red', padding='5px'))
grid[0,0] = choices
grid[0,1] = amount_choice
grid[0,2] = reset_inventory
grid[1,0] = random_choice
grid[1,1] = add_choice
grid[1,2] = recommendation

v = widgets.VBox(children=(output_label, inventory_output), disabled=False,  layout=widgets.Layout(align='center'))
display(grid)
display(v)

In [ ]:
%%html
<h1>Below is a graph that displays the data of the current recommended items. </h1>

In [ ]:
# plot_out
plot_out_1

In [ ]:
%%html
<p style="border: 5px solid red; font-size: 28px; text-align:center; padding: 15px; line-height: 1.1"> We are anxiously awaiting the completion of the Android app to fully utilize the recommendation program and ensure better profits and more sales and happy customers!!!</p>

In [ ]:
%%html
<div style="text-align:center; border: 5px dotted purple; padding: 15px; font-size: 30px">CopyRight &copy; Catch22Software 2022
<br><br>
</div>

In [ ]:
ball = open('Images/curveball.jpg','rb')
image = ball.read()
ball_show = widgets.Image(value=image,format='jpg',width=200,height=200,layout=widgets.Layout(align='center',justify_items='center'))
display(ball_show)